# Quick start
This section is a quick start guide that will show the basic functionality of McStasScript. It assumes the user is already familiar with McStas itself, if this is not the case, it is recommended to start with the tutorial which can serve as an introduction to both McStas and McStasScript.

## Importing the package
McStasScript needs to be imported into the users python environment.

In [ ]:
import mcstasscript as ms

## Creating the first instrument object
Now the package can be used. Start with creating a new instrument, just needs a name. For a McXtrace instrument use McXtrace_instr instead.

In [ ]:
instrument = ms.McStas_instr("first_instrument")

### Finding a component
The instrument object loads the available McStas components, so it can show these in order to help the user.

In [ ]:
instrument.available_components()

In [ ]:
instrument.available_components("sources")

### Adding the first component
McStas components can be added to the instrument, here we add a source and ask for help on the parameters.

In [ ]:
source = instrument.add_component("source", "Source_simple")
source.show_parameters()

### Set parameters
The parameters of the component object are adjustable directly through the attributes of the object.

In [ ]:
source.xwidth = 0.03
source.yheight = 0.03
source.lambda0 = 3
source.dlambda = 2.2
source.dist = 5
source.focus_xw = 0.01
source.focus_yh = 0.01
print(source)

### Instrument parameters
It is possible to add instrument parameters that can be adjusted when running the simulation or adjusted using the widget interface.

In [ ]:
wavelength = instrument.add_parameter("wavelength", value=3, comment="Wavelength in AA")
source.lambda0 = wavelength
source.dlambda = "0.1*wavelength"
print(source)

### Inserting a sample component
A sample component is added as any other component, but here we place it relative to the source. A SANS sample is used, it focuses to a detector (chosen with target_index) with a width of focus_xw and height of focus_yh.

In [ ]:
sample = instrument.add_component("sans_sample", "Sans_spheres")

In [ ]:
sample.set_AT(5, RELATIVE=source)
sample.set_parameters(R=120, xwidth=0.01, yheight=0.01, zdepth=0.01,
                      target_index=1, focus_xw=0.5, focus_yh=0.5)
print(sample)

### Adding a monitor
The monitor can be placed relative to the sample, and even use the attributes from the sample to define its size so that the two always match. When setting a filename, it has to be a string also in the generated code, so use double quotation marks as shown here.

In [ ]:
PSD = instrument.add_component("PSD", "PSD_monitor")
PSD.set_AT([0, 0, 5], RELATIVE=sample)
PSD.set_parameters(xwidth=sample.focus_xw, yheight=sample.focus_yh, filename='"PSD.dat"')

## Setting up the simulation
The instrument now contains a source, a sample and a monitor, this is enough for a simple demonstration.

In [ ]:
instrument.show_parameters()

In [ ]:
instrument.set_parameters(wavelength=4)
instrument.settings(ncount=2E6)

### Performing the simulation
In order to start the simulation the *backengine* method is called. If the simulation is successful, the data will be returned, otherwise the method returns None.

In [ ]:
data = instrument.backengine()

## Plot the data
The data can be plotted with the *make_sub_plot* function from the plotter module.

In [ ]:
ms.make_sub_plot(data, log=True)

### Access the data
The data is a list of McStasData objects and can be accessed directly.

In [ ]:
print(data)

It is possible to search through the data list with the *name_search* function to retrieve the desired data object. 

In [ ]:
PSD_data = ms.name_search("PSD", data)

The intensities can then be accessed directly, along with Error and Ncount. 

In [ ]:
PSD_data.Intensity

Metadata is also available as a dict.

In [ ]:
info_dict = PSD_data.metadata.info
for field, info in info_dict.items():
    print(field, ":", info)